# Script for TuRBO PSxPS using TACO data

In [2]:
!pip install astropy
!pip install git+https://github.com/naught101/sobol_seq@v0.2.0#egg=sobol_seq

     |████████████████████████████████| 10.3 MB 757 kB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11684 sha256=d390e6a6d6233b59b2dc26fb86b43153bc67c8192c2faea1394f5e7f0ce8179d
  Stored in directory: /root/.cache/pip/wheels/22/cf/f4/600b7619db8e0ce75023fc14145fc432a54522b7b7b4778ecf
Successfully built bayesian-optimization
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
  Cloning https://github.com/naught101/sobol_seq (to revision v0.2.0) to /tmp/pip-install-6n9kyau3/sobol-seq
  Created wheel for sobol-seq: filename=sobol_seq-0.2.0-py3-none-any.whl size=8350 sha256=b9fcd1d8111c9fb95915e9f1d101d78be69cdba11c1f85fdbd476a369c0e55be
  Stored in directory: /tmp/pip-ephem-wheel-cache-mmnxomic/wheels/99/8e/fe/458a2b11f274d622f7c6da0c5c264105ff8fe9ffde8352fe84
Successfully 

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time as timer
import scipy
import sobol_seq
import torch, gpytorch

from astropy.timeseries import LombScargle
import sys
sys.path.append('TuRBO/')
from turbo import Turbo1, TurboM
from turbo.utils import from_unit_cube, latin_hypercube, to_unit_cube
import frequencies, mixed_modes_utils
from tqdm import tqdm_notebook as tqdm

In [6]:
mos_df = pd.read_csv('/tables/Mosser_2018.dat', delim_whitespace=True)
mos_df = mos_df.rename(columns={'q': 'mos_q', 'dpi': 'mos_dpi', 'eps_g': 'mos_eps_g', 'rot': 'mos_rot',
                               'dnu': 'mos_dnu'})
mos_df

,KIC,numax,e_numax,mos_dnu,e_dnu,mos_dpi,e_dpi,mos_q,e_q,mos_eps_g,e_eps_g,mos_rot,e_rot
0,1026084,45.80,0.57,4.45,0.04,250.58,0.15,0.35,0.04,0.032,0.051,40,10
1,1026326,94.91,1.18,8.87,0.05,74.55,0.02,0.12,0.02,0.068,0.049,160,12
2,1027337,74.15,0.90,6.98,0.04,69.67,0.03,0.12,0.02,-0.057,0.094,240,14
3,1161618,33.71,0.51,4.08,0.04,329.95,0.16,0.19,0.03,-0.043,0.049,45,11
4,1162746,27.18,0.48,3.81,0.04,279.19,0.12,0.30,0.04,0.020,0.061,65,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,11550492,93.64,1.14,8.70,0.04,74.86,0.05,0.13,0.02,0.027,0.104,390,17
368,11913545,116.75,1.38,10.18,0.05,77.68,0.02,0.14,0.02,0.204,0.033,270,12
369,11968334,140.68,1.59,11.41,0.05,77.91,0.04,0.13,0.02,0.239,0.044,370,15
370,12008916,160.78,1.87,12.90,0.06,79.45,0.10,0.15,0.02,0.017,0.109,440,23


# Preamble code for search

In [47]:
#' It is assumed that DeltaNu is in μHz
def DeltaPi1_from_DeltaNu_RGB(DeltaNu):
    # Compute Period spacing (in s) from deltanu
    return 60 + 1.7*DeltaNu

def Lor_model(pds, peak):
    return peak.height / (1 + ((pds.frequency.values - peak.frequency)/peak.linewidth)**2)

def sinc2_model(pds, peak):
    deltanu = np.mean(np.diff(pds.frequency.values))
    return peak.height * np.sinc((pds.frequency.values - peak.frequency)/deltanu)**2

def fit_model(pds, peaks):

    model = np.ones_like(pds.frequency.values)

    for i in range(len(peaks)):
        if np.isfinite(peaks.linewidth.iloc[i]):
            model += Lor_model(pds, peaks.iloc[i,])
        else:
            model += sinc2_model(pds, peaks.iloc[i, ])
    return model

In [48]:
from scipy.stats import chi2


def psps_significance(DPi1, q):

    params = {'calc_l0': True, # Compute radial mode properties
            'calc_l2': True, # Compute l=2 mode properties
            'calc_l3': False, # Don't need to calculate l=3 theoretical freqs
            'calc_nom_l1': True, # Compute nominal l=1 p-mode properties
            'calc_mixed': False, # Don't compute mixed modes (as not needed)
            'calc_rot': False, # Don't compute rotation
            'DPi1': DPi1,
            'coupling': q,
            'eps_g': 0.0, # Epsilon_g isn't needed for computation of tau due to chosen formulation of zeta
            'l': 1, # Mixed modes are dipole mixed modes
            }
    freqs(params)
    new_frequency, tau, zeta = mixed_modes_utils.stretched_pds(pds_l023_removed.frequency.values, 
                                                               freqs.zeta)

    f = np.arange(1/(400.), 1/(20.), 0.1/tau.max())
    ls = LombScargle(tau, pds_l023_removed.power.values)
    PSD_LS = ls.power(f)

    noise = np.median(PSD_LS) / (1 - 1/9)**3
   
    # Cut down period and power arrays to search range if given
    cut_PSD_LS = PSD_LS[(1/f > 20) & (1/f < 400)]
    cut_f = f[(1/f > 20) & (1/f < 400)]

    # Report significance level of highest peak in search area
    sig_prob = chi2.cdf((cut_PSD_LS/noise)[np.argmin(np.abs((1/cut_f) - DPi1))], df=2) # try 
    return sig_prob



def prepare_l1_peaks(peaks: pd.DataFrame, summary: pd.DataFrame,
                     AIC_cut: [float] = 0.0, height_cut: [float] = 0.0) -> pd.DataFrame:
    """
    Extract the mixed modes from the peaks dataframe.
    
    Parameters
    ----------
    peaks: pd.DataFrame
        Dataframe containing the detected peaks and parameters.
        
    summary: pd.DataFrame
        Dataframe containing the global stellar information.
    
    AIC_cut: Optional[float] = 0.0
        Cut to make in the Akaike Information Criterion if desired.
        
    height_cut: Optional[float] = 0.0
        Cut to make in the mode height if desired.
        
    Outputs
    -------
    pd.DataFrame
        Dataframe containing the mixed mode peaks and associated mode parameters.
    """
    peaks['x'] = ((peaks['frequency'] % summary['DeltaNu'].values - summary['eps_p'].values) / summary['DeltaNu'].values) % 1
    # Don't want to include any modes near l=0 or 2s, this is why this and the step in the next cell is performed.
    x_range = [(np.minimum(np.min(peaks.loc[peaks['l'] == 0, 'x']), np.min(peaks.loc[peaks['l'] == 2, 'x'])) - 0.05) % 1,
               (np.maximum(np.max(peaks.loc[peaks['l'] == 0, 'x']), np.max(peaks.loc[peaks['l'] == 2, 'x'])) + 0.05) % 1]
    
    l1_peaks = peaks.loc[(peaks.l == 1) | ~np.isfinite(peaks.l), ]
    l1_peaks['x'] = ((l1_peaks['frequency'] % summary['DeltaNu'].values - summary['eps_p'].values) / summary['DeltaNu'].values) % 1
    if x_range[0] < x_range[1]:
        l1_peaks = l1_peaks.loc[(l1_peaks['x'] < x_range[1]) | (l1_peaks['x'] > x_range[0]), ]
    else:
        l1_peaks = l1_peaks.loc[(l1_peaks['x'] > x_range[1]) & (l1_peaks['x'] < x_range[0]), ]

    l1_peaks = l1_peaks.loc[(l1_peaks['height'] > height_cut), ]
    l1_peaks = l1_peaks.loc[(l1_peaks['AIC'] > AIC_cut), ]

    return l1_peaks

# Automating the whole process

In [73]:
## Defining helper functions

from joblib import Parallel, delayed


def psxps(summary,dpi, q, pds_l023_removed, low_bound,
               up_bound):    
    
    lower_psps, upper_psps = low_bound, up_bound
    freqs = frequencies.Frequencies(frequency=pds_l023_removed.frequency.values,
                                numax=summary.numax.values, 
                                delta_nu=summary.DeltaNu.values if np.isfinite(summary.DeltaNu.values) else None, 
                                epsilon_p=summary.eps_p.values if np.isfinite(summary.eps_p.values) else None,
                                alpha=summary.alpha.values if np.isfinite(summary.alpha.values) else None)
    
    params = {'calc_l0': True, # Compute radial mode properties
                'calc_l2': True, # Compute l=2 mode properties
                'calc_l3': False, # Don't need to calculate l=3 theoretical freqs
                'calc_nom_l1': True, # Compute nominal l=1 p-mode properties
                'calc_mixed': False, # Don't compute mixed modes (as not needed)
                'calc_rot': False, # Don't compute rotation
                'DPi1': dpi, #DeltaPi1_from_DeltaNu_RGB(dnu),
                'coupling': q,
                'eps_g': 0.0, # Epsilon_g isn't needed for computation of tau due to chosen formulation of zeta
                'l': 1, # Mixed modes are dipole mixed modes
                }
    # Make computation - in our case this is for the computation of zeta
    freqs(params)
    # Compute tau from the zeta value just computed
    new_frequency, tau, zeta = mixed_modes_utils.stretched_pds(pds_l023_removed.frequency.values, 
                                                               freqs.zeta)


    f = np.arange(1/(upper_psps), 1/(lower_psps), 0.1/tau.max())
    ls = LombScargle(tau, pds_l023_removed.power.values)
    PSD_LS = ls.power(f)
    noise = np.median(PSD_LS) / (1 - 1/9)**3

    return 1/f, PSD_LS/noise


class PSXPS:
    def __init__(self, init_dpi, low_bound, up_bound, factor=1):
        
        self.factor = factor
        init_dpi = init_dpi*self.factor
        
        self.lb = np.array([0.8*init_dpi, 0])
        self.ub = np.array([1.2*init_dpi, 0.6])
        self.low_bound = low_bound
        self.up_bound = up_bound
        
    def __call__(self, x):
        assert len(x) == 2
        assert x.ndim == 1
        assert np.all(x <= self.ub) and np.all(x >= self.lb)
        
        DPi1, q = x[0], x[1]
        
        params = {'calc_l0': True, 
                'calc_l2': True, 
                'calc_l3': False, 
                'calc_nom_l1': True,
                'calc_mixed': False, 
                'calc_rot': False,
                'DPi1': DPi1,
                'coupling': q,
                'eps_g': 0.0,
                'l': 1,
                }
        freqs(params)
        new_frequency, tau, zeta = mixed_modes_utils.stretched_pds(pds_l023_removed.frequency.values, 
                                                                   freqs.zeta)

        f = np.arange(1/(self.up_bound), 1/(self.low_bound), 0.1/tau.max())
        ls = LombScargle(tau, pds_l023_removed.power.values)
        PSD_LS = ls.power(f)
        noise = np.median(PSD_LS) / (1 - 1/9)**3
        cut_PSD_LS = PSD_LS[(1/f > self.low_bound) & (1/f < self.up_bound)]
        cut_f = f[(1/f > self.low_bound) & (1/f < self.up_bound)]
        
        return -(cut_PSD_LS/noise)[np.argmin(np.abs((1/cut_f) - (1./self.factor)*DPi1))]      

In [74]:
kic_array = []

In [75]:
plt.close()

In [77]:
from tqdm import tqdm_notebook as tqdm
import os, sobol_seq

for i, iz in tqdm(enumerate(mos_df['KIC'].values), total = len(mos_df['KIC'].values)):
    
    evol_state = int(mos_df['mos_dpi'].values[i] < 120)
    
    mos_dpi, mos_q = mos_df['mos_dpi'].values[i], mos_df['mos_q'].values[i]
          
    if iz in kic_array:
        continue
        
        
    print('KIC %d' %(iz))
     
    # Load up the summary file, power spectrum and detected peaks
    
    basefolder = '/data/00%d/' %iz
    
    if not os.path.exists(basefolder):
        basefolder = '/data/0%d/' %iz
    if not os.path.exists(basefolder):
        print('KIC %d not peakbagged!' %iz)
        continue

    summary = pd.read_csv(basefolder+'summary.csv')
    pds = pd.read_csv(basefolder+'pds_bgr.csv')
    peaks = pd.read_csv(basefolder+'peaksMLE.csv')
    
    if evol_state != 0: #RGB
        low_bound, up_bound, dpi_guess, q_guess = 20, 150, DeltaPi1_from_DeltaNu_RGB(summary.DeltaNu.values), 0.2
    else:
        low_bound, up_bound, dpi_guess, q_guess = 150, 400, 300, 0.3

    prepped_l1_peaks = prepare_l1_peaks(peaks, summary=summary, AIC_cut=0)

    # Only keep pds around oscillations
    pds = pds.loc[abs(pds['frequency'].values - summary['numax'].values) < 3 * summary['sigmaEnv'].values, ]
    peaks = peaks.loc[abs(peaks.frequency.values - summary.numax.values) < 3*summary.sigmaEnv.values, ]

    # Split the peaks in the l=0,2,3 peaks (which have been already identified)
    # and the rest, which should hopefully be unidentified l=3
    l023_peaks = peaks.loc[(peaks.l == 0) | (peaks.l == 2) | (peaks.l == 3), ]
    l0_peaks = peaks.loc[(peaks.l==0), ]
    l1_peaks = peaks.loc[(peaks.l == 1) | (np.isfinite(peaks.l) == False)]
    l2_peaks = peaks.loc[(peaks.l==2), ]
    
    # Divide the data through by the model of the l=0,2 modes
    pds_l023_removed = pds.assign(power = pds.power / fit_model(pds, l023_peaks))
    
    try:
        freqs = frequencies.Frequencies(frequency=pds_l023_removed.frequency.values,
                                        numax=summary.numax.values, 
                                        delta_nu=summary.DeltaNu.values if np.isfinite(summary.DeltaNu.values) else None, 
                                        epsilon_p=summary.eps_p.values if np.isfinite(summary.eps_p.values) else None,
                                        alpha=summary.alpha.values if np.isfinite(summary.alpha.values) else None)
    except:
        plt.clf()
        plt.close()
        continue
        
            
    init_period, init_psps = psxps(summary = summary, dpi=dpi_guess,
                                   q=q_guess, pds_l023_removed=pds_l023_removed, low_bound=low_bound,
               up_bound=up_bound)

    mos_period, mos_psps = psxps(summary=summary, dpi=mos_dpi,
                                   q=mos_q, pds_l023_removed=pds_l023_removed, low_bound=low_bound,
               up_bound=up_bound)
    
    
    ## Run Bayesian optimizer ##
    
    init_dpi = init_period[np.argmax(init_psps)]
    
    if len(init_psps) == 0:
        plt.clf()
        plt.close()
        continue 
    
    if init_dpi < 30:
        factor = 3.
    elif init_dpi < 50:
        factor = 2
    else:
        factor = 1
    fnc = PSXPS(init_dpi=init_dpi, low_bound=low_bound,
               up_bound=up_bound, factor=factor)
   
    turbo1 = Turbo1(
    f=fnc,  # Handle to objective function
    lb=fnc.lb,  # Numpy array specifying lower bounds
    ub=fnc.ub,  # Numpy array specifying upper bounds
    n_init=100,  # Number of initial bounds from an Latin hypercube design
    max_evals = 1000,  # Maximum number of evaluations
    batch_size=25,  # How large batch size TuRBO uses
    verbose=True,  # Print information from each batch
    use_ard=True,  # Set to true if you want to use ARD for the GP kernel
    max_cholesky_size=2000,  # When we switch from Cholesky to Lanczos
    n_training_steps=50,  # Number of steps of ADAM to learn the hypers
    min_cuda=64,  # Run on the CPU for small datasets
    device="cuda",  # "cpu" or "cuda"
    dtype="float64",  # float64 or float32
    ) 
        
    init_time = timer.time()
    turbo1.optimize()
    print('Time Taken to Optimize PSxPS: ', timer.time()-init_time)
    
    X = turbo1.X  # Evaluated points
    fX = turbo1.fX  # Observed values
    # X.shape, fX.shape
    ind_best = np.argmin(fX)
    f_best, x_best = fX[ind_best], X[ind_best, :]
    best_dpi, best_q= x_best[0], x_best[1]
    
    # Create PSxPS of final estimate
    
    final_period, final_psps = psxps(summary=summary, dpi=best_dpi,
                                   q=best_q,
                                     pds_l023_removed=pds_l023_removed, low_bound=low_bound,
               up_bound=up_bound)
    
    fig = plt.figure(figsize=(25, 10))
    ax1, ax2, ax3 = fig.add_subplot(311), fig.add_subplot(312), fig.add_subplot(313)
     
    ax1.plot(pds.frequency.values, pds.power.values, label='Original')
    ax1.plot(pds.frequency.values, pds_l023_removed.power.values, label='Dipole Only')
    ax1.legend()
    ax1.set_xlabel('Frequency ($\\mu$Hz)')
    ax1.set_ylabel('Power (SNR)')
    
    ax2.plot(final_period, final_psps, label='Bayes Opt PSxPS')
    ax2.plot(mos_period, mos_psps, label='Mosser 2018 PSxPS')
    
    ax2.axvline(x=mos_dpi, c='k', ls='--', label='Mosser Estimate')
    ax2.set_ylabel('Power')
    
    
    ## Sobol Sampling ##
    
    import sobol_seq
    
    dpi1_lower = fnc.lb
    dpi1_upper = fnc.ub
    pbounds = {'mos_dpi': (fnc.lb[0], fnc.ub[0]),
             'mos_q': (fnc.lb[1], fnc.ub[1])} # (0.0, 0.6)
    
    samp_sob = sobol_seq.i4_sobol_generate(2, 2**15)
    samp_sob[:,0] = samp_sob[:,0]*(pbounds['mos_dpi'][1] - pbounds['mos_dpi'][0]) + pbounds['mos_dpi'][0]
    samp_sob[:,1] = samp_sob[:,1]*(pbounds['mos_q'][1] - pbounds['mos_q'][0]) + pbounds['mos_q'][0] # scaling the Sobol sampled points
    idx_sob = np.argsort(samp_sob[:,0])
    samp_sob = samp_sob[idx_sob]
    samples = to_unit_cube(samp_sob, fnc.lb, fnc.ub)
    
    with torch.no_grad(), gpytorch.settings.max_cholesky_size(turbo1.max_cholesky_size):
        X_cand_torch = torch.tensor(samples).cuda()
        y_cand = turbo1.best_gp.likelihood(turbo1.best_gp(X_cand_torch))

    mu = y_cand.mean.data.cpu().numpy()
        
    ax3.scatter(samp_sob[:,0], samp_sob[:,1], c=mu);
    ax3.set_xlabel(r'DPi1 (s)');
    ax3.set_ylabel(r'Coupling');
    
    
    ax3.scatter(best_dpi, best_q, marker='*', s=150, linewidths=4, color='magenta', zorder=3) 
    ax3.scatter(mos_dpi, mos_q, c='k', marker='o', s=75)
    ax3.set_xlim(fnc.lb[0], fnc.ub[0])
    ax3.set_ylim(fnc.lb[1], fnc.ub[1]);
    ax2.set_xlim(fnc.lb[0], fnc.ub[0])
    ax2.axvline(x=best_dpi, c='r', ls='--', label='Best BayesOpt Estimate')
    ax2.legend()

    
    plt.tight_layout(h_pad=0)
    plt.savefig('/results/images/PSxPS_images/%d.png' %iz)
    plt.clf()
    plt.close()
    
    kic_array.append(iz)

    
    with open('/results/tables/auto_dPi1_images_TACO_PSxPS.txt', 'a') as writer:
        writer.write(str(int(iz))+','+str(np.round(best_dpi,3))+','+str(np.round(best_q, 3))+','\
                     +str(mos_dpi)+','+str(mos_q)+'\n')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/372 [00:00<?, ?it/s]

KIC 1726291
Using dtype = torch.float64 
Using device = cuda


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Starting from fbest = -6.126
125) New best: -6.516 Solution: [78.7830187  0.1587196]
150) New best: -9.221 Solution: [83.0446587   0.13663433]
175) New best: -13.07 Solution: [82.58443377  0.15444042]
300) New best: -13.13 Solution: [82.51862246  0.15545434]
325) New best: -13.27 Solution: [82.4932082  0.1579998]
350) New best: -13.35 Solution: [82.61251776  0.15756334]
475) Restarting with fbest = -13.35
Starting from fbest = -5.192
900) New best: -13.36 Solution: [82.57726664  0.15763902]
950) Restarting with fbest = -13.36
Starting from fbest = -8.323
Time Taken to Optimize PSxPS:  34.56466603279114
KIC 2998532
Using dtype = torch.float64 
Using device = cuda


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Starting from fbest = -15.08
150) New best: -20.71 Solution: [75.07484119  0.12121524]
175) New best: -21.46 Solution: [74.77005012  0.11049692]
200) New best: -23.68 Solution: [74.81054919  0.12682022]
250) New best: -23.83 Solution: [74.86065144  0.12211297]
350) New best: -24.12 Solution: [74.8260667   0.12360921]
425) Restarting with fbest = -24.12
Starting from fbest = -11.15
800) New best: -24.13 Solution: [74.84033992  0.12400436]
825) Restarting with fbest = -24.13
Starting from fbest = -23.53
Time Taken to Optimize PSxPS:  34.15035152435303


In [ ]:
#1726291, 2998532